### Start/loading in

In [1]:
# import sys
import time
import numpy as np
import pathlib
import pandas as pd
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, TimestampType, StringType, IntegerType, FloatType, DateType
from pyspark.sql.functions import pandas_udf, PandasUDFType, date_trunc, col
import pyspark.pandas as ps

# spark.stop()

In [2]:
conf = pyspark.SparkConf().setAll([\
    ('spark.app.name', 'ReduceData')])
spark = SparkSession.builder.config(conf=conf)\
    .getOrCreate()

spark.version

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/10 08:15:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


'3.3.1'

### Schemas

In [3]:
glucose_data_schema=StructType([StructField('NumId', IntegerType(), True),
                                        StructField('PatientId', StringType(), True),
                                        StructField('Value', FloatType(), True),
                                        StructField('GlucoseDisplayTime', TimestampType(), True),
                                        StructField('GlucoseDisplayTimeRaw', StringType(), True),
                                        StructField('GlucoseDisplayDate', DateType(), True)])

In [4]:
"""replicate step1: read in the data"""
startTime = time.time()
data_location = "/cephfs/train_test_val/train_set"
allPaths = [str(x) for x in list(pathlib.Path(data_location).glob('*.parquet')) if 'part-00' in str(x)][0]

pyspark_glucose_data = spark.read \
                       .schema(glucose_data_schema) \
                       .format('parquet') \
                       .load(allPaths)
pyspark_glucose_data = pyspark_glucose_data.withColumn("GlucoseDisplayTime",
                                                       date_trunc("minute",
                                                       col("GlucoseDisplayTime")))

'''drop duplicate datetimes for each patient'''
window = Window.partitionBy('GlucoseDisplayTime','PatientId').orderBy('tiebreak')
df = (df
 .withColumn('tiebreak', monotonically_increasing_id())
 .withColumn('rank', rank().over(window))
 .filter(col('rank') == 1).drop('rank','tiebreak')
)

pyspark_glucose_data=pyspark_glucose_data.orderBy("PatientId",
                                                  "GlucoseDisplayTime",
                                                  ascending=True)

print(time.time()-startTime)

20.191951513290405


In [5]:
test_df = pyspark_glucose_data.toPandas()
test_df = test_df.drop(columns=['PatientId','GlucoseDisplayTimeRaw','GlucoseDisplayDate'])

/home/jovyan/glucose-data-analysis/glucose_venv/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


In [58]:
min_max = test_df.groupby('NumId')\
            .agg({'GlucoseDisplayTime' : ['min', 'max']})

merge_df = pd.DataFrame(columns=['GlucoseDisplayTime', 'NumId'])
for idx, row in min_max.iterrows():
    # print(test_df[test_df['NumId'] == idx].info(),"\n")
    #grab all potential dates in range

    date_df = pd.DataFrame(pd.date_range(row[0], row[1], freq='5min'), columns=['GlucoseDisplayTime'])                              
    date_df['NumId'] = idx

    # merge dates with big pypsark df
    merged = test_df[test_df['NumId'] == idx]\
            .merge(date_df, how='outer', on=['GlucoseDisplayTime', 'NumId'])\
            .sort_values(by=['GlucoseDisplayTime', 'Value'], na_position='last')

    merged['TimeLag'] = np.concatenate((merged['GlucoseDisplayTime'].iloc[0],\
                                        np.array(merged['GlucoseDisplayTime'].iloc[:-1].values)), axis=None)\
                        .astype('datetime64[ns]')
    print(merged.info(),"\n")

    merged['Diff'] = (merged['TimeLag'] - merged['GlucoseDisplayTime']).dt.seconds

    len_merged = len(merged)

    # get all index of rows with diff less than 5 mins, add 1 to remove next row, 
    # dont include last row to delete
    indexes_to_remove = [x for x in merged[merged['Diff'] < 300].index + 1 if x < len_merged]

    if len(indexes_to_remove) > 0:
        merged = merged.drop(indexes_to_remove)

    merged = merged.drop(columns=['TimeLag','Diff'])

    # its ready freddy for some interpoletty
    merge_df = pd.concat([merge_df,merged])

    break
    
# merge_df
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150016 entries, 0 to 58814
Data columns (total 4 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   NumId               150016 non-null  int64         
 1   Value               58815 non-null   float32       
 2   GlucoseDisplayTime  150016 non-null  datetime64[ns]
 3   TimeLag             150016 non-null  datetime64[ns]
dtypes: datetime64[ns](2), float32(1), int64(1)
memory usage: 5.2 MB
None 

27
<class 'pandas.core.frame.DataFrame'>
Int64Index: 149989 entries, 1 to 58814
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   NumId               149989 non-null  int64         
 1   Value               58788 non-null   float32       
 2   GlucoseDisplayTime  149989 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float32(1), int64(1)
memory usage: 4.0 MB


In [51]:
temp = [i - 1 for i in indexes_to_remove] + indexes_to_remove
temp.sort()
merged.loc[temp]

,NumId,Value,GlucoseDisplayTime
0,149,112.0,2022-02-01 00:06:00
1,149,110.0,2022-02-01 00:11:00
46156,149,143.0,2022-11-17 04:42:00
46157,149,112.0,2022-11-17 04:42:00
46165,149,215.0,2022-11-17 05:27:00
46166,149,150.0,2022-11-17 05:27:00
46169,149,212.0,2022-11-17 05:42:00
46170,149,149.0,2022-11-17 05:42:00
46177,149,178.0,2022-11-17 06:42:00
46178,149,180.0,2022-11-17 06:47:00


[0,
 1,
 46156,
 46157,
 46165,
 46166,
 46169,
 46170,
 46177,
 46178,
 46178,
 46179,
 46189,
 46190,
 46191,
 46192,
 46193,
 46194,
 46196,
 46197,
 46197,
 46198,
 46200,
 46201,
 46205,
 46206,
 46206,
 46207,
 46211,
 46212,
 46212,
 46213,
 46218,
 46219,
 46221,
 46222,
 46228,
 46229,
 46230,
 46231,
 46234,
 46235,
 46237,
 46238,
 46239,
 46240,
 46242,
 46243,
 46244,
 46245,
 46252,
 46253,
 46254,
 46255]